In [1]:
import pandas as pd

In [2]:
last_week = pd.read_excel(r"C:\Users\pa173\Documents\SQL Excel Extracts\Weekly Producer\January\Jan 15 - Jan 21 2023.xlsx", sheet='Export Worksheet', index=False)

In [3]:
all_parent = pd.read_excel(r"C:\Users\pa173\Documents\Communications\All Producer with Parent Rollup.xlsx", index=False)

In [4]:
rank_list = pd.read_excel(r"C:\Users\pa173\Documents\Communications\Wkly Working - Rank and List.xlsx", index=False)

In [5]:
last_report = pd.read_excel(r"C:\Users\pa173\Documents\Communications\Weekly NB and RW Apps and FQ.xlsx", sheet_name='By Producer Weekly Counts', index=False, skiprows= 9)
last_report = last_report.drop(index=0)

In [6]:
nb_rw_pivot = pd.read_excel(r"C:\Users\pa173\Documents\Communications\Weekly NB and RW Apps and FQ.xlsx", sheet_name='Pivot', index=False, skiprows= 6)

In [7]:
def modify_cols(df):
    df.columns=df.columns.str.lower()
    df.columns=df.columns.str.strip()
    df.columns=df.columns.str.replace(" ", "_")
    return df


In [8]:
last_week=modify_cols(last_week)
all_parent=modify_cols(all_parent)
rank_list=modify_cols(rank_list)
last_report=modify_cols(last_report)
nb_rw_pivot=modify_cols(nb_rw_pivot)

merging last weeks data with all producers

In [9]:
parent_week=pd.merge(left=last_week, right=all_parent, left_on='sub_business_unit_number', right_on='business_unit_number', how='left')

In [10]:
parent_week.shape

(471, 15)

In [11]:
producer_pivot=parent_week.pivot_table(index=['parent_name'], values=['count(*)'], aggfunc='sum')
#producer_pivot = parent_week.pivot_table(index='parent_name', values='count(*)', columns=['parent_name','count(*)'], aggfunc='sum').sort_values(['count(*)'], ascending=False)

In [12]:
producer_pivot.shape

(457, 1)

In [13]:
rank_list.shape

(1012, 3)

let's merge our producer pivot table with the weekly rank

In [14]:
producer_rank=pd.merge(left=producer_pivot, right=rank_list, left_on='parent_name', right_on='producer', how='left')

In [15]:
producer_rank.shape

(457, 4)

cleaning nb and rw pivots

In [16]:
nb_pivot = nb_rw_pivot[['row_labels', 'unnamed:_8']]
nb_pivot=nb_pivot.rename(columns={'row_labels':'nb_labels','unnamed:_8':'nb_total'})

In [17]:
rw_pivot = nb_rw_pivot[['row_labels.1', 'unnamed:_19']]
rw_pivot=rw_pivot.rename(columns={'row_labels.1':'rw_labels','unnamed:_19':'rw_total'})
rw_pivot=rw_pivot.dropna()

In [18]:
nb_pivot.shape, rw_pivot.shape

((205, 2), (104, 2))

clean last weeks report

In [19]:
last_report = last_report.iloc[:, 1:17]
last_report.iloc[:,4:16] = last_report.iloc[:, 4:16].apply(pd.to_numeric)
last_report= last_report.rename(columns = {'weekly_counts_by_week': 'producer'})

merge parent pivot with last weeks report

In [20]:
new_report=pd.merge(last_report, producer_rank, left_on='producer', right_on='producer', how='left')

In [21]:
new_report.shape

(1015, 19)

merge new report with nb and rw tables

In [22]:
new_report=pd.merge(new_report, nb_pivot, left_on='producer', right_on='nb_labels', how='left')

In [23]:
new_report.shape

(1015, 21)

In [24]:
new_report=pd.merge(new_report, rw_pivot, left_on='producer', right_on='rw_labels', how='left')

In [25]:
new_report.shape

(1015, 23)

In [26]:
new_report=new_report.fillna(0)

In [27]:
new_report['incl']=new_report['incl'].replace(0,'N')

converting incl column as category since it's a boolean column

In [28]:
new_report.columns

Index(['producer', 'change_quotes', 'change_nb_apps', 'change_rw_apps',
       'quotes', 'nb_apps', 'rw_apps', 'quotes.1', 'nb_apps.1', 'rw_apps.1',
       'quotes.2', 'nb_apps.2', 'rw_apps.2', 'quotes.3', 'nb_apps.3',
       'rw_apps.3', 'count(*)', 'rank', 'incl', 'nb_labels', 'nb_total',
       'rw_labels', 'rw_total'],
      dtype='object')

In [29]:
new_report['incl']=new_report['incl'].astype('category')

0    Y
1    Y
2    Y
3    Y
4    Y
Name: incl, dtype: category
Categories (2, object): [N, Y]

updating file with this weeks data

In [31]:
true_count = (new_report['count(*)'].isin(new_report['quotes'])).sum()
false_count = (new_report['count(*)'].isin(new_report['quotes'])==False).sum()
print("Number of true values:", true_count)
print("Number of false values:", false_count)


Number of true values: 1015
Number of false values: 0


In [32]:
new_report['updated_quotes']=new_report['count(*)']
new_report['updated_nb_apps']=new_report['nb_total']
new_report['updated_rw_apps']=new_report['rw_total']

In [34]:
new_report['change_quotes']=new_report['count(*)'] - new_report['quotes.1']
new_report['change_nb_apps']=new_report['nb_total'] - new_report['nb_apps.1']
new_report['change_rw_apps']=new_report['rw_total'] - new_report['rw_apps.1']

updating only the columns we need

In [39]:
#new_report=new_report[['producer', 'change_quotes', 'change_nb_apps', 'change_rw_apps', 
#        'updated_quotes', 'updated_nb_apps', 'updated_rw_apps',
#       'quotes', 'nb_apps', 'rw_apps', 'quotes.1', 'nb_apps.1', 'rw_apps.1',
#       'quotes.2', 'nb_apps.2', 'rw_apps.2', 'quotes.3']]

In [40]:
new_report.head()

,producer,change_quotes,change_nb_apps,change_rw_apps,quotes,nb_apps,rw_apps,quotes.1,nb_apps.1,rw_apps.1,...,count(*),rank,incl,nb_labels,nb_total,rw_labels,rw_total,updated_quotes,updated_nb_apps,updated_rw_apps
0,ROADWAY INSURANCE INC #1,-46.0,-8.0,6.0,469,60.0,48.0,515.0,68.0,42.0,...,469.0,1.0,Y,ROADWAY INSURANCE INC #1,60.0,ROADWAY INSURANCE INC #1,48.0,469.0,60.0,48.0
1,EZ INSURANCE GROUP LLC,-47.0,0.0,-1.0,361,36.0,17.0,408.0,36.0,18.0,...,361.0,2.0,Y,EZ INSURANCE GROUP LLC,36.0,EZ INSURANCE GROUP LLC,17.0,361.0,36.0,17.0
2,KEYSTONE INSURANCE AGENCY INC,-6.0,4.0,6.0,75,24.0,15.0,81.0,20.0,9.0,...,75.0,3.0,Y,KEYSTONE INSURANCE AGENCY INC,24.0,KEYSTONE INSURANCE AGENCY INC,15.0,75.0,24.0,15.0
3,PINNING INS AGENCY INC #1,18.0,0.0,-6.0,231,27.0,5.0,213.0,27.0,11.0,...,231.0,4.0,Y,PINNING INS AGENCY INC #1,27.0,PINNING INS AGENCY INC #1,5.0,231.0,27.0,5.0
4,JAMES BROWN INS AGENCY INC,74.0,-2.0,3.0,297,27.0,11.0,223.0,29.0,8.0,...,297.0,5.0,Y,JAMES BROWN INS AGENCY INC,27.0,JAMES BROWN INS AGENCY INC,11.0,297.0,27.0,11.0


In [41]:
new_report.to_excel(r"C:\Users\pa173\Documents\Python\Weekly Report\new_report.xlsx", index=False)